In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import urllib.request

url = 'http://ergast.com/downloads/f1db_csv.zip'
filehandle, _ = urllib.request.urlretrieve(url)
zf = zipfile.ZipFile(filehandle, 'r')

results_df = pd.read_csv(zf.open('results.csv'))
drivers_df = pd.read_csv(zf.open('drivers.csv'))
lap_times_df = pd.read_csv(zf.open('lap_times.csv'))
pit_stops_df = pd.read_csv(zf.open('pit_stops.csv'))

In [2]:
raceId = lap_times_df.raceId.drop_duplicates().iloc[-1]
race_df = lap_times_df[lap_times_df["raceId"] == raceId].copy()
pit_stop_df = pit_stops_df[pit_stops_df.raceId == raceId]

for driverId in pit_stop_df.driverId.drop_duplicates():
    for lap in pit_stop_df[pit_stop_df.driverId == driverId].lap:
        race_df.loc[(race_df.driverId == driverId) & (race_df.lap == lap + 1),'milliseconds'] = np.nan
        mean_value = race_df.loc[(race_df.driverId == driverId) & (race_df.lap.isin([lap, lap + 1, lap + 2]))].milliseconds.mean()
        race_df.loc[(race_df.driverId == driverId) & (race_df.lap == lap + 1),'milliseconds'] = mean_value

phase_data = []
        
for driverId in race_df.driverId.drop_duplicates():
    driver_lap_time_df = race_df[race_df.driverId == driverId].copy()
    driver_lap_time_df["delta_milliseconds"] = driver_lap_time_df.milliseconds - driver_lap_time_df.milliseconds.shift()
    driver_lap_time_df["delta_milliseconds"] = driver_lap_time_df["delta_milliseconds"].fillna(0)
    driver_info = drivers_df[drivers_df["driverId"] == driverId]
    driver_name = driver_info["forename"] + ' ' + driver_info["surname"]
    phase_data.append([driver_name.iloc[0], driver_lap_time_df.delta_milliseconds.mean()])
    
pd.DataFrame(phase_data, columns = ['Driver name', 'Phase earned']).sort_values("Phase earned", ascending=False)

,Driver name,Phase earned
1,Max Verstappen,-56.333333
0,Lewis Hamilton,-57.409091
2,Lance Stroll,-85.246154
5,Alexander Albon,-99.123077
4,Sergio Pérez,-104.415385
7,Pierre Gasly,-129.876923
6,Carlos Sainz,-147.553846
9,Lando Norris,-154.030769
10,Sebastian Vettel,-154.446154
8,Charles Leclerc,-154.868421
